# Mediapipe handlandmark_영상 녹화 시작 및 저장

In [1]:
#<1.upper_left 학습용 영상 저장>
import cv2
import mediapipe as mp
import numpy as np
import time, os
from datetime import datetime

# actions = ['upper_left', 'upper_pront', 'upper_right', 'lower_left', 'lower_front', 'lower_right']
actions = ['upper_left'] #윗니 레이블 = 0, 아랫니 레이블 = 1
seq_length = 30
secs_for_action = 30

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# 카메라 오픈
cap = cv2.VideoCapture(0)

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

# 영상 저장 관련 설정
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 비디오 코덱 설정
out = cv2.VideoWriter('output_upper.avi', fourcc, 30.0, (640, 480))  # 저장할 영상 파일 설정

while cap.isOpened():
    for idx, action in enumerate(actions):
        data = [] 
        
        ret, img_upper = cap.read()

        if not ret:
            break

        img_upper = cv2.flip(img_upper, 1)
        
        cv2.putText(img_upper, f'Waiting for collecting {action.upper()} action...', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.imshow('img_upper', img_upper)
        cv2.waitKey(3000)
        
        start_time = time.time()

        while time.time() - start_time < secs_for_action:
            ret, img_upper = cap.read()

            img_upper = cv2.flip(img_upper, 1)
            img_upper = cv2.cvtColor(img_upper, cv2.COLOR_BGR2RGB)
            #손 랜드마크 검출
            result = hands.process(img_upper)
            img_upper = cv2.cvtColor(img_upper, cv2.COLOR_RGB2BGR)        
    
            if result.multi_hand_landmarks is not None:
                for hand_landmarks in result.multi_hand_landmarks:
                    # 손 각도 계산
                    joint = np.zeros((21, 4))
                    for j, lm in enumerate(hand_landmarks.landmark):
                        joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

                    # Compute angles between joints
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3]  # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3]  # Child joint
                    v = v2 - v1  # [20, 3]
                    # Normalize v
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    # Get angle using arcos of dot product
                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))  # [15,]

                    angle = np.degrees(angle)  # Convert radian to degree
            
                    angle_label = np.array([angle], dtype=np.float32)
                    angle_label = np.append(angle_label, idx)

                    d = np.concatenate([joint.flatten(), angle_label])

                    data.append(d)

                    mp_drawing.draw_landmarks(img_upper, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            #프레임을 비디오에 추가
            out.write(img_upper)
            
            cv2.imshow('img_upper', img_upper)
            if cv2.waitKey(1) == ord('q'):
                break
                
        data = np.array(data)
        print(action, data.shape)
        np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

# 비디오 녹화 종료
out.release()

# 영상 프레임을 NumPy 배열로 변환하고 reshape
#data = np.array(frames)
#data = data.reshape(-1, 224, 224, 3)


upper_left (553, 100)


KeyboardInterrupt: 

In [2]:
#<2.lower_left 학습용 영상 저장>
import cv2
import mediapipe as mp
import numpy as np
import time, os
from datetime import datetime

# actions = ['upper_left', 'upper_pront', 'upper_right', 'lower_left', 'lower_front', 'lower_right']
actions = ['lower_left'] #윗니 레이블 = 0, 아랫니 레이블 = 1
seq_length = 30
secs_for_action = 30

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# 카메라 오픈
cap = cv2.VideoCapture(0)

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

# 영상 저장 관련 설정
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 비디오 코덱 설정
out = cv2.VideoWriter('output_lower.avi', fourcc, 30.0, (640, 480))  # 저장할 영상 파일 설정

while cap.isOpened():
    for idx, action in enumerate(actions):
        data = [] 
        
        ret, img_lower = cap.read()

        if not ret:
            break

        img_lower = cv2.flip(img_lower, 1)
        
        cv2.putText(img_lower, f'Waiting for collecting {action.lower()} action...', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.imshow('img_lower', img_lower)
        cv2.waitKey(3000)
        
        start_time = time.time()

        while time.time() - start_time < secs_for_action:
            ret, img_lower = cap.read()

            img_lower = cv2.flip(img_lower, 1)
            img_lower = cv2.cvtColor(img_lower, cv2.COLOR_BGR2RGB)
            #손 랜드마크 검출
            result = hands.process(img_lower)
            img_lower = cv2.cvtColor(img_lower, cv2.COLOR_RGB2BGR)        
    
            if result.multi_hand_landmarks is not None:
                for hand_landmarks in result.multi_hand_landmarks:
                    # 손 각도 계산
                    joint = np.zeros((21, 4))
                    for j, lm in enumerate(hand_landmarks.landmark):
                        joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

                    # Compute angles between joints
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3]  # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3]  # Child joint
                    v = v2 - v1  # [20, 3]
                    # Normalize v
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    # Get angle using arcos of dot product
                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:]))  # [15,]

                    angle = np.degrees(angle)  # Convert radian to degree
            
                    angle_label = np.array([angle], dtype=np.float32)
                    angle_label = np.append(angle_label, idx)

                    d = np.concatenate([joint.flatten(), angle_label])

                    data.append(d)

                    mp_drawing.draw_landmarks(img_lower, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            #프레임을 비디오에 추가
            out.write(img_lower)
            
            cv2.imshow('img_lower', img_lower)
            if cv2.waitKey(1) == ord('q'):
                break
                
        data = np.array(data)
        print(action, data.shape)
        np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

# 비디오 녹화 종료
out.release()

lower_left (604, 100)


KeyboardInterrupt: 

# CNN 모델 train

In [3]:
import cv2
import numpy as np
import time, os
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from datetime import datetime

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [4]:
#영상 불러오기 #라벨 데이터 생성 부분이 올바르지 않음.
def load_data():
    #영상 데이터를 불러오기
    cap1 = cv2.VideoCapture('output_upper.avi')
    cap2 = cv2.VideoCapture('output_lower.avi')
    frames_upper = []
    frames_lower = []
    while True:
        #각각의 프레임 읽기
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()
        if not ret1 or not ret2:
            break
        if ret1:
            frame1_resized = cv2.resize(frame1, (320, 240))  # 이미지 크기 조정
            frames_upper.append(frame1_resized)
        if ret2:
            frame2_resized = cv2.resize(frame2, (320, 240))  # 이미지 크기 조정
            frames_lower.append(frame2_resized)
    cap1.release()
    cap2.release()
    
    frames = frames_upper + frames_lower  # upper와 lower 프레임을 합침
    #라벨 데이터 생성
    label1 = np.zeros(len(frames_upper), dtype=int)  # upper 영상 전체를 0으로 라벨링
    label2 = np.ones(len(frames_lower), dtype=int)  # lower 영상 전체를 1로 라벨링
    #두 라벨 데이터를 병합
    labels = np.concatenate((label1, label2))

    return np.array(frames), labels

# 데이터 전처리 함수
def preprocess_data(frames):
    # 이미지 크기 조정 및 정규화
    processed_frames = []
    for frame in frames:
        frame = cv2.resize(frame, (224, 224))
        frame = frame / 255.0  # 픽셀 값을 0에서 1 사이로 정규화
        processed_frames.append(frame)
    return np.array(processed_frames)
# 모델 생성 함수
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu')) #두 번째 마지막 dense layer 연결
    model.add(Dense(1, activation='sigmoid'))
    return model

In [5]:
# 필요한 모듈 가져오기
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import SGD

# 데이터 로드
frames, labels = load_data()

# 데이터 전처리
processed_frames = preprocess_data(frames)

#데이터 분할(검증 데이터만 분리)
X_train = processed_frames
y_train = labels

# 모델 생성
model = create_model()

# 모델 컴파일
optimizer = SGD(learning_rate=0.001)  # SGD 옵티마이저를 사용하고 learning rate를 0.001로 설정
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    #binary_crossentropy: 2-class의 제한된 형태에서 사용. 마지막에 dense = 2로 해줘야 함.

# 모델 학습
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)  # validation_split을 사용하여 검증 데이터를 분리합니다.





Epoch 1/10


28/28 [==============================] - 30s 939ms/step - loss: 0.5992 - accuracy: 0.7849 - val_loss: 1.9237 - val_accuracy: 0.1486
Epoch 2/10
28/28 [==============================] - 25s 896ms/step - loss: 0.0199 - accuracy: 0.9955 - val_loss: 1.8041 - val_accuracy: 0.4820
Epoch 3/10
28/28 [==============================] - 25s 893ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 2.8027 - val_accuracy: 0.3739
Epoch 4/10
28/28 [==============================] - 25s 893ms/step - loss: 4.3680e-04 - accuracy: 1.0000 - val_loss: 0.6278 - val_accuracy: 0.8063
Epoch 5/10
28/28 [==============================] - 25s 897ms/step - loss: 1.3532e-04 - accuracy: 1.0000 - val_loss: 1.2855 - val_accuracy: 0.7072
Epoch 6/10
28/28 [==============================] - 25s 900ms/step - loss: 5.9291e-05 - accuracy: 1.0000 - val_loss: 1.1259 - val_accuracy: 0.7297
Epoch 7/10
28/28 [==============================] - 25s 910ms/step - loss: 3.2565e-05 - accuracy: 1.0000 - val_loss: 1.2344 -

In [6]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (1110, 224, 224, 3)
y_train shape: (1110,)


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 173056)            0

In [10]:
#콜백함수
#콜백: 학습 중 모델의 동작을 제어하고 모니터링
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

history = model.fit(
    X_train, y_train, 
    epochs=100,
    callbacks=[
        ModelCheckpoint('models/model.cnn', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto'),
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
    ]
)

Epoch 1/100
35/35 [==============================] - 31s 875ms/step - loss: 0.0697 - accuracy: 0.9838 - lr: 0.0010
Epoch 2/100
35/35 [==============================] - 30s 844ms/step - loss: 0.0097 - accuracy: 0.9982 - lr: 0.0010
Epoch 3/100
35/35 [==============================] - 30s 843ms/step - loss: 5.3254e-05 - accuracy: 1.0000 - lr: 0.0010
Epoch 4/100
35/35 [==============================] - 30s 845ms/step - loss: 9.4068e-06 - accuracy: 1.0000 - lr: 0.0010
Epoch 5/100
35/35 [==============================] - 29s 842ms/step - loss: 2.4711e-06 - accuracy: 1.0000 - lr: 0.0010
Epoch 6/100
35/35 [==============================] - 30s 845ms/step - loss: 1.7598e-06 - accuracy: 1.0000 - lr: 0.0010
Epoch 7/100
35/35 [==============================] - 30s 850ms/step - loss: 1.5600e-06 - accuracy: 1.0000 - lr: 0.0010
Epoch 8/100
35/35 [==============================] - 30s 857ms/step - loss: 1.4270e-06 - accuracy: 1.0000 - lr: 0.0010
Epoch 9/100
35/35 [==============================] - 30s

# 성능평가: CNN 모델 test

In [11]:
#<test 영상 저장>
import cv2
import mediapipe as mp
import numpy as np
import time, os
from datetime import datetime
#from mediapipe.tasks.python.audio import audio_classifier

#actions = ['upper_left', 'upper_front', 'upper_right', 'lower_left', 'lower_front', 'lower_right'']
actions = ['upper_left', 'lower_left']
seq_length = 30
secs_for_action = 30

#MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    #max_min_hands=1, 
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

#카메라 오픈
cap = cv2.VideoCapture(0)

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

#영상 저장 관련 설정
fourcc = cv2.VideoWriter_fourcc(*'XVID') #비디오 코덱 설정
out = cv2.VideoWriter('output_test.avi', fourcc, 30.0, (640, 480)) #저장할 영상 파일 설정

while cap.isOpened():
    for idx, action in enumerate(actions):
        data = []
        
        ret, img = cap.read()
        
        if not ret:
            break
        
        img = cv2.flip(img, 1)
        
        cv2.putText(img, f'Waiting for collecting {action.upper()} action...', org=(10,30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
        cv2.imshow('img', img)
        cv2.waitKey(3000)
        
        start_time = time.time()
        
        while time.time() - start_time < secs_for_action:
            ret, img = cap.read()
            
            img = cv2.flip(img,1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            #손 랜드마크 검출
            result = hands.process(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            
            if result.multi_hand_landmarks is not None:
                for hand_landmarks in result.multi_hand_landmarks:
                    #손 각도 계산
                    joint = np.zeros((21,4))
                    for j, lm in enumerate(hand_landmarks.landmark):
                        joint[j] = [lm.x, lm.y, lm.z, lm.visibility]
                        
                    # Compute angles between joints
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                    v = v2 - v1 # [20, 3]
                    # Normalize v
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    # Get angle using arcos of dot product
                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

                    angle = np.degrees(angle) # Convert radian to degree

                    angle_label = np.array([angle], dtype=np.float32)
                    angle_label = np.append(angle_label, idx)

                    d = np.concatenate([joint.flatten(), angle_label])

                    data.append(d)

                    mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS)
          
            #프레임을 비디오에 추가
            out.write(img)
            
            cv2.imshow('img', img)
            if cv2.waitKey(1) == ord('q'):
                break
        
        data = np.array(data)
        print(action, data.shape)
        np.save(os.path.join('dataset', f'raw_{action}_{created_time}'), data)

#비디오 녹화 종료
out.release()

upper_left (358, 100)
lower_left (378, 100)


KeyboardInterrupt: 

In [12]:
#'output_test'영상 불러오기 
def load_data():
    #영상 데이터를 불러오기
    cap = cv2.VideoCapture('output_test.avi')
    frames = []
    while True:
        #프레임 읽기
        ret, frame = cap.read()

        if not ret:
            break
        if ret:
            frame_resized = cv2.resize(frame, (320, 240))  # 이미지 크기 조정
            frames.append(frame_resized)
    cap.release()

    return np.array(frames)

In [13]:
# 데이터 전처리 함수
def preprocess_data(frames):
    # 이미지 크기 조정 및 정규화
    processed_frames = []
    for frame in frames:
        frame = cv2.resize(frame, (224, 224))
        frame = frame / 255.0  # 픽셀 값을 0에서 1 사이로 정규화
        processed_frames.append(frame)
    return np.array(processed_frames)

#데이터 전처리
processed_frames = preprocess_data(frames)
#test = processed_frames

#모델 평가(필요하다면 추가)
test_loss, test_acc = model.evaluate(processed_frames)
print("테스트 정확도:", test_acc)

35/35 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
테스트 정확도: 0.0
